### Simplex
---

Construção do algoritmo Simplex de duas fases:

#### Fase I: Busca de uma solução factível 
A função `find_basis` faz a verificação de "factibilidade" do problema, caso positivo, fornece uma base para iniciar a busca de uma solução ótima.

#### Fase II: Busca de uma solução ótima
A função `Simplex.calculate`* busca uma solução ótima para o problema com base numa solução factível inicial. 


\* Essa função também é usada na fase anterior para resolver o "problema auxiliar" que criamos para achar uma base.

In [1]:
import numpy as np
from numpy.linalg import inv

def find_basis(A, c, b):
        """
        Gera a base, determinando se o problema é factível.
        """
        
        print("Buscando uma base...")
        
        # Criamos i novas variáveis
        i = A.shape[0]
        A_exp = np.append(A, np.identity(i), axis=1)
        
        # Atualizando c e criando a base inicial
        c = np.zeros((A_exp.shape[1], 1))
        c[-i:] = 1
        B = np.where(c == 1)[0]
        
        # Solução inicial
        pinv = np.linalg.pinv(A)
        x = np.dot(pinv, b)
        
        # Calculando simplex para o problema "artificial"
        s = Simplex(A_exp, c, x, b, maximum=False, B=B)
        x = s.calculate()
        
        # Checando solução factível: variáveis novas devem estar zeradas!
        if x[i:].any() == 0:
            B = np.where(x[:i] == 0)
            print(">> Base encontrada: B == {}".format(B))
            
            return B
        
        else:
            print("Problema não é factível :(")


class Simplex:

    """
    Instancia o metodo Simplex.

    Parameters
    ----------
    A : np.matrix
        Matriz de restricoes do PL
    c : np.array
        Vetor de custos do PL.
    x : np.array
        Vetor de solução inicial do PL.
    b : np.array
        Vetor de resposta do PL.
    maximum : bool
        Define problema de maximização (True) ou minimização (False). Defautl True.
    w : int
        Custo independente do PL.
    B : np.array, optional
        Base do PL.
    steps : int, optional
        Número máximo de iterações. Default 100.

    Return
    ------
    x : np.array
        Solucao otima do PL (nx1).
    """
    def __init__(self, A, c, x, b, maximum=True, w=0, B=np.nan, steps=500):

        self.A = A
        self.c = c
        self.x = x
        self.b = b
        self.w = w
        self.steps = steps
        
        if type(B) == np.ndarray:
            self.B = B
            
        if maximum:
            self.operator = max
        else:
            self.operator = min
            
    def canonical_form(self):
        """
        Passa o PL para a forma canônica: (i) A_B deve ser identidade, (ii) c_B = 0
        """
        
        if not np.all(self.c[self.B] != 0) or not np.all(self.A[:, self.B] != np.identity(len(self.B))):
            
            A_Binv = np.linalg.inv(self.A[:, self.B])
            self.y = A_Binv.T.dot(self.c[self.B])
            self.c = self.c.T - self.y.T.dot(self.A).ravel()
            self.c = self.c.T
            self.w = self.w + np.dot(self.y.T, self.b)
            
            self.A = A_Binv.dot(self.A)
            self.b = A_Binv.dot(self.b)
            
            print('A: \n', self.A)
            print('b: \n', self.b)
            print('c: \n', self.c)
            print('w: \n', self.w)
            
            print(">> Problema canonizado! Indo para (2)...")
            
        else:
            print(">> Problema ja na forma canonica! Indo para (2)...")
    
    def step(self):
        
        x = np.zeros(self.A.shape[1])
        x[self.B] = np.linalg.solve(self.A[:, self.B], b).T
        
        var_basicas = np.where(x != 0)
        var_nao_basicas = np.where(x == 0)
        
        # Encontrar tamanho do passo
        rows = self.b.shape[0]
        
        t_set = np.array([self.b[i]/self.A[i, self.k] for i in range(rows)])
        t = t_set[t_set > 0] # t > 0
        
        if self.operator:
            t = min(t) # problema de maximizacao
        else:
            t = max(t) # problema de minimizacao
            
        self.r = np.where(t_set == t)[0][0]

        # Dar o passo escolhido
        x_aux = self.b - t*self.A[:, self.k]
        x_aux = np.array(x_aux.tolist()) # "flatten": shape (3,1) -> (3,)
        
        self.x = np.zeros((self.c.shape[0], 1))
        self.x[self.k] = t
        self.x[var_basicas] = x_aux
        print('>> x: \n{}'.format(self.x))

    def calculate(self):

        print(">>> Iniciando Simplex!")
        self.N = list(set(range(self.A.shape[1])) - set(self.B))
        print(self.N)

        i = 0
        while i < self.steps:
        #while self.c[self.N].any() > 0:
            print("\n\n>>> Iteração {}:".format(i))
            print("\n1. Reescrevendo o PL na forma canonica para a base B...\n")
            self.canonical_form()
            
            print("\n2. Teste de otimalidade (c_N <= 0):")
            if np.all(self.c[self.N] <= 0):
                print(">>> Processo terminado!")
                return self.x
            print("Não passou no teste...\n")
            print(">> c_N: \n{}".format(self.c[self.N]))
            print(">> c_B: \n{}".format(self.c[self.B]))

            print("\n3. Seleciona k em N tal que c[k] > 0...")
            # self.k = np.argmax(self.c)
            self.k = np.where(self.c > 0)[0][0] # primeiro valor não nulo
            print(">> k: {}".format(self.k))

            print("\n4. Teste de unboundness (A[:,k] <= 0):")
            if np.all(self.A[:, self.k] <= 0):
                (">>> Processo terminado!")
                return self.x
            print("Não passou no teste...\n")
            print(">> A[:,k]: \n{}".format(self.A[:, self.k]))
            
            print("\n5. Passo Simplex...")
            self.step()

            print("\n6. Atualizando a base...")
            print(self.r)
            print(self.k)
            self.B = np.append(self.B.copy(), self.k)
            self.B = np.delete(self.B.copy(), self.r)
            
            self.N = list(set(range(self.A.shape[1])) - set(self.B))
            print(">> B: \n{}".format(self.B))
            
            i += 1

        print("Processo terminado!")
        return self.x

### Exemplos
---

#### Fase I + Fase II

In [2]:
# Formato do PL: max{z = cx + w : Ax = b, x >= 0}
A = np.matrix([[1,5,2,1], [-2,-9,0,3]])
c = np.array([1,2,-1,3]).reshape(4,1)
b = np.array([7,-13]).reshape(2,1)
#B = np.array([2,3,4])

# Fase I: Testa se a solução é factível
sx = find_basis(A, c, b)

# Fase II: Acha a solução ótima
#s = Simplex(A, c, x, b, B=B)
#s.calculate()

Buscando uma base...
>>> Iniciando Simplex!
[0, 1, 2, 3]


>>> Iteração 0:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[ 1.  5.  2.  1.  1.  0.]
 [-2. -9.  0.  3.  0.  1.]]
b: 
 [[  7.]
 [-13.]]
c: 
 [[ 1.]
 [ 4.]
 [-2.]
 [-4.]
 [ 0.]
 [ 0.]]
w: 
 [[-6.]]
>> Problema canonizado! Indo para (2)...

2. Teste de otimalidade (c_N <= 0):
Não passou no teste...

>> c_N: 
[[ 1.]
 [ 4.]
 [-2.]
 [-4.]]
>> c_B: 
[[0.]
 [0.]]

3. Seleciona k em N tal que c[k] > 0...
>> k: 0

4. Teste de unboundness (A[:,k] <= 0):
Não passou no teste...

>> A[:,k]: 
[[ 1.]
 [-2.]]

5. Passo Simplex...
>> x: 
[[6.5]
 [0. ]
 [0. ]
 [0. ]
 [0.5]
 [0. ]]

6. Atualizando a base...
1
0
>> B: 
[4 0]


>>> Iteração 1:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[ 0.   0.5  2.   2.5  1.   0.5]
 [ 1.   4.5  0.  -1.5  0.  -0.5]]
b: 
 [[0.5]
 [6.5]]
c: 
 [[ 0. ]
 [-0.5]
 [-2. ]
 [-2.5]
 [ 0. ]
 [ 0.5]]
w: 
 [[0.5]]
>> Problema canonizado! Indo para (2)...

2. Teste de otimalidade (c_N <=

#### Somente Fase II (base já fornecida quando instanciamos o algoritmo)

In [3]:
# Formato do PL: max{z = cx + w : Ax = b, x >= 0}
A = np.matrix([[1,1,1,0,0], [2,1,0,1,0], [-1,1,0,0,1]])
c = np.array([2,3,0,0,0]).reshape(5,1)
x = np.array([0,0,6,10,4]).reshape(5,1)
b = np.array([6,10,4]).reshape(3,1)
B = np.array([2,3,4])

# Fase I: Testa se a solução é factível
#find_basis(A, c, b)

# Fase II: Acha a solução ótima
s = Simplex(A, c, x, b, B=B)
sol = s.calculate()

>>> Iniciando Simplex!
[0, 1]


>>> Iteração 0:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[ 1.  1.  1.  0.  0.]
 [ 2.  1.  0.  1.  0.]
 [-1.  1.  0.  0.  1.]]
b: 
 [[ 6.]
 [10.]
 [ 4.]]
c: 
 [[2.]
 [3.]
 [0.]
 [0.]
 [0.]]
w: 
 [[0.]]
>> Problema canonizado! Indo para (2)...

2. Teste de otimalidade (c_N <= 0):
Não passou no teste...

>> c_N: 
[[2.]
 [3.]]
>> c_B: 
[[0.]
 [0.]
 [0.]]

3. Seleciona k em N tal que c[k] > 0...
>> k: 0

4. Teste de unboundness (A[:,k] <= 0):
Não passou no teste...

>> A[:,k]: 
[[ 1.]
 [ 2.]
 [-1.]]

5. Passo Simplex...
>> x: 
[[5.]
 [0.]
 [1.]
 [0.]
 [9.]]

6. Atualizando a base...
1
0
>> B: 
[2 4 0]


>>> Iteração 1:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[ 0.   0.5  1.  -0.5  0. ]
 [ 0.   1.5  0.   0.5  1. ]
 [ 1.   0.5  0.   0.5  0. ]]
b: 
 [[1.]
 [9.]
 [5.]]
c: 
 [[ 0.]
 [ 2.]
 [ 0.]
 [-1.]
 [ 0.]]
w: 
 [[10.]]
>> Problema canonizado! Indo para (2)...

2. Teste de otimalidade (c_N <= 0):
Não passou no teste

In [4]:
sol

array([[0.],
       [1.],
       [0.],
       [3.],
       [5.]])

In [5]:
A*sol

matrix([[1.],
        [4.],
        [6.]])